# Lab Activity - Working with Mistral

### Tasneem Al Sheikh - 60104251

_________________________

In [1]:
pip install mistralai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Load API key and helper function

In [3]:
import os
os.environ["MISTRAL_API_KEY"]="CZ54uDhpNVIrfJTvnjflHs4LuiSU4XHr"
print(f"MISTRAL_API_KEY: {os.environ.get('MISTRAL_API_KEY')}")

MISTRAL_API_KEY: CZ54uDhpNVIrfJTvnjflHs4LuiSU4XHr


In [4]:
api_key = os.getenv("MISTRAL_API_KEY")

In [5]:
import os
from mistralai import Mistral, UserMessage
def mistral(user_message, model="mistral-small-latest", is_json=False):
    model = "mistral-large-latest"
    client = Mistral(api_key=api_key)
    messages = [
    UserMessage(content=user_message),
    ]
    
    chat_response = client.chat.complete(
    model=model,
    messages=messages,
    )
    return chat_response.choices[0].message.content

In [6]:
mistral("hello, what can you do?")

'Hello! I can help with a wide range of tasks and provide information on various topics. Here are some things I can do:\n\n1. **Answer Questions**: I can provide information on a broad array of subjects, including science, history, technology, and more.\n2. **Explain Concepts**: I can help clarify complex ideas or theories.\n3. **Provide Recommendations**: Whether you need book suggestions, movie recommendations, or travel tips, I can assist.\n4. **Help with Writing**: I can help with writing essays, reports, or even creative writing tasks.\n5. **Language Translation**: I can translate text between multiple languages.\n6. **Math Problems**: I can help solve mathematical problems or explain mathematical concepts.\n7. **Coding Assistance**: I can assist with programming questions in various languages, such as Python, JavaScript, and more.\n8. **Learning Resources**: I can suggest resources for learning new skills or subjects.\n9. **General Advice**: I can offer advice on various topics, 

## Classification

In [7]:
prompt = """
You are a bank customer service bot.
Your task is to assess customer intent and categorize customer
inquiry after <<<>>> into one of the following predefined categories:
card arrival
change pin
exchange rate
country support
cancel transfer
charge dispute
If the text doesn't fit into any of the above categories,
classify it as:
customer service
You will only respond with the predefined category.
Do not provide explanations or notes.
###
Here are some examples:
Inquiry: How do I know if I will get my card, or if it is lost? I am concerned about the delivery process and would like to ensure that I will receive my ca
Category: card arrival
Inquiry: I am planning an international trip to Paris and would like to inquire about the current exchange rates for Euros as well as any associated fees fo
Category: exchange rate
Inquiry: What countries are getting support? I will be traveling and living abroad for an extended period of time, specifically in France and Germany, and w
Category: country support
Inquiry: Can I get help starting my computer? I am having difficulty starting my computer, and would appreciate your expertise in helping me troubleshoot th
Category: customer service
###
<<<
Inquiry: {inquiry}
>>>
Category:
"""

# Correct spelling and grammar
response = mistral(f"Please correct the spelling and grammar of this prompt and return a text that is the same prompt, with the spelling and grammar fixed: {prompt}")
print(response)

# Try out the model
print(mistral(response.format(inquiry="I am inquiring about the availability of your cards in the EU")))


You are a bank customer service bot.
Your task is to assess customer intent and categorize the customer inquiry after <<<>>> into one of the following predefined categories:
card arrival
change pin
exchange rate
country support
cancel transfer
charge dispute
If the text doesn't fit into any of the above categories,
classify it as:
customer service
You will only respond with the predefined category.
Do not provide explanations or notes.
###
Here are some examples:
Inquiry: How do I know if I will get my card, or if it is lost? I am concerned about the delivery process and would like to ensure that I will receive my card.
Category: card arrival
Inquiry: I am planning an international trip to Paris and would like to inquire about the current exchange rates for Euros as well as any associated fees for foreign transactions.
Category: exchange rate
Inquiry: What countries are supported? I will be traveling and living abroad for an extended period of time, specifically in France and Germany, 

## Information Extraction with JSON Mode



In [8]:
medical_notes = """
A 60-year-old male patient, Mr. Johnson, presented with symptoms
of increased thirst, frequent urination, fatigue, and unexplained
weight loss. Upon evaluation, he was diagnosed with diabetes,
confirmed by elevated blood sugar levels. Mr. Johnson's weight
is 210 lbs. He has been prescribed Metformin to be taken twice daily
with meals. It was noted during the consultation that the patient is
a current smoker.
"""

prompt = f"""
Extract information from the following medical notes:
{medical_notes}
Return json format with the following JSON schema:
{{
"age": {{
"type": "integer"
}},
"gender": {{
"type": "string",
"enum": ["male", "female", "other"]
}},
"diagnosis": {{
"type": "string",
"enum": ["migraine", "diabetes", "arthritis", "acne"]
}},
"weight": {{
"type": "integer"
}},
"smoking": {{
"type": "string",
"enum": ["yes", "no"]
}}
}}
"""

response = mistral(prompt, is_json=True)
print(response)


```json
{
  "age": 60,
  "gender": "male",
  "diagnosis": "diabetes",
  "weight": 210,
  "smoking": "yes"
}
```


## Personalization



In [9]:
email = """
Dear mortgage lender,
What's your 30-year fixed-rate APR, how is it compared to the 15-year
fixed rate?
Regards,
Anna
"""

prompt = f"""
You are a mortgage lender customer service bot, and your task is to
create personalized email responses to address customer questions.
Answer the customer's inquiry using the provided facts below. Ensure
that your response is clear, concise, and directly addresses the
customer's question. Address the customer in a friendly and
professional manner. Sign the email with "Lender Customer Support."
# Facts
30-year fixed-rate: interest rate 6.403%, APR 6.484%
20-year fixed-rate: interest rate 6.329%, APR 6.429%
15-year fixed-rate: interest rate 5.705%, APR 5.848%
10-year fixed-rate: interest rate 5.500%, APR 5.720%
7-year ARM: interest rate 7.011%, APR 7.660%
5-year ARM: interest rate 6.880%, APR 7.754%
3-year ARM: interest rate 6.125%, APR 7.204%
30-year fixed-rate FHA: interest rate 5.527%, APR 6.316%
30-year fixed-rate VA: interest rate 5.684%, APR 6.062%
# Email
{email}
"""

response = mistral(prompt)
print(response)


Dear Anna,

Thank you for reaching out to us regarding our mortgage rates.

Our 30-year fixed-rate mortgage has an Annual Percentage Rate (APR) of 6.484%. In comparison, our 15-year fixed-rate mortgage has an APR of 5.848%.

If you have any further questions or need additional information, please feel free to contact us.

Best regards,

Lender Customer Support


## Summarization Task



In [10]:
newsletter = """
European AI champion Mistral AI unveiled new large language models and formed an alliance with Microsoft.
What’s new: Mistral AI introduced two closed models, Mistral Large and Mistral Small (joining Mistral Medium, which debuted quietly late last year). Microsoft i
Model specs: The new models’ parameter counts, architectures, and training methods are undisclosed. Like the earlier, open source Mistral 7B and Mixtral 8x7B, t
Mistral Large achieved 81.2 percent on the MMLU benchmark, outperforming Anthropic’s Claude 2, Google’s Gemini Pro, and Meta’s Llama 2 70B, though falling short
Both models are fluent in French, German, Spanish, and Italian. They’re trained for function calling and JSON-format output.
Microsoft’s investment in Mistral AI is significant but tiny compared to its $13 billion stake in OpenAI and Google and Amazon’s investments in Anthropic, which
Mistral AI and Microsoft will collaborate to train bespoke models for customers including European governments.
Behind the news: Mistral AI was founded in early 2023 by engineers from Google and Meta. The French government has touted the company as a home-grown competitor
Yes, but: Mistral AI’s partnership with Microsoft has divided European lawmakers and regulators. The European Commission, which already was investigating Micros
Why it matters: The partnership between Mistral AI and Microsoft gives the startup crucial processing power for training large models and greater access to pote
We’re thinking: Mistral AI has made impressive progress in a short time, especially relative to the resources at its disposal as a startup. Its partnership with
"""

prompt = f"""
You are a commentator. Your task is to write a report on a newsletter.
When presented with the newsletter, come up with interesting questions to ask,
and answer each question.
Afterward, combine all the information and write a report in the markdown
format.
# Newsletter:
{newsletter}
# Instructions:
## Summarize:
In clear and concise language, summarize the key points and themes
presented in the newsletter.
## Interesting Questions:
Generate three distinct and thought-provoking questions that can be
asked about the content of the newsletter. For each question:
- After "Q: ", describe the problem
- After "A: ", provide a detailed explanation of the problem addressed
in the question.
- Enclose the ultimate answer in <>.
## Write a analysis report
Using the summary and the answers to the interesting questions,
create a comprehensive report in Markdown format.
"""

response = mistral(prompt)
print(response)


# Newsletter Report

## Summary

Mistral AI, a European AI company, has introduced two new large language models: Mistral Large and Mistral Small. These models join Mistral Medium, which was quietly released last year. The new models are proficient in multiple European languages and are trained for function calling and JSON-format output. Mistral Large has shown impressive performance on the MMLU benchmark, surpassing models from competitors like Anthropic, Google, and Meta.

Microsoft has invested in Mistral AI, forming a strategic alliance. This partnership aims to train bespoke models for customers, including European governments. The collaboration has, however, sparked debate among European lawmakers and regulators, particularly due to ongoing investigations into Microsoft by the European Commission. Despite the controversy, the alliance provides Mistral AI with crucial processing power and broader market access, cementing its position as a significant player in the AI landscape.

